In [1]:
import pandas as pd

# 1. Read dataset
print("[1] Read DATASET")
df = pd.read_csv("https://storage.googleapis.com/dqlab-dataset/retail_raw_test.csv", low_memory=False)
print("Dataset:\n", df.head())
print("Info:\n", df.info())

[1] Read DATASET
Dataset:
    order_id    order_date customer_id  ... quantity item_price product_value
0   1730350  Dec 11, 2019      '13447  ...      '24    '113000        1374.0
1   1677490  Jul 31, 2019          '0  ...       '1   '1164000        1370.0
2   1704211  Oct 18, 2019      '16128  ...      '12    '747000        1679.0
3   1679695  Aug 07, 2019      '16225  ...       '6    '590000        1708.0
4   1679080  Aug 05, 2019          '0  ...       '2    '740000        1201.0

[5 rows x 9 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       5000 non-null   int64  
 1   order_date     5000 non-null   object 
 2   customer_id    5000 non-null   object 
 3   city           3802 non-null   object 
 4   province       3802 non-null   object 
 5   brand          4995 non-null   object 
 6   quantity       5000 non-nu

In [ ]:
df.describe()

,order_id,product_value
count,5.000000e+03,4995.000000
mean,1.680654e+06,2166.392593
std,3.894844e+04,1156.094360
min,1.612372e+06,2.000000
25%,1.645560e+06,1246.500000
50%,1.682586e+06,2104.000000
75%,1.715150e+06,3132.500000
max,1.742950e+06,4204.000000


In [2]:
# 2. Change data type
print("\n[2] change data type")
df["customer_id"] = df["customer_id"].apply(lambda x: x.split("'")[1]).astype("int64")
df["quantity"] = df["quantity"].apply(lambda x : x.split("'")[1]).astype("int64")
df["item_price"] = df["item_price"].apply(lambda x : x.split("'")[1]).astype("int64")
print("    Tipe data:\n", df.dtypes)


[2] change data type
    Tipe data:
 order_id           int64
order_date        object
customer_id        int64
city              object
province          object
brand             object
quantity           int64
item_price         int64
product_value    float64
dtype: object


In [3]:
# 3. Transform "product_value" so that the shape is uniform with the format "PXXXX", assign it to a new column "product_id", and drop the column "product_value", if there is nan replace it with "unknown"
print("\n[3] TRANSFORM product_value to product_id")
# Make a function
import math
def impute_product_value(val):
    if math.isnan(val):
        return "unknown"
    else:
        return 'P' + '{:0>4}'.format(str(val).split('.')[0])
# Make a column "product_id"
df["product_id"] = df["product_value"].apply(lambda x: impute_product_value(x))
# Drop a column "product_value"
df.drop(["product_value"], axis=1, inplace=True)
# checking dataset
print(df.head())



[3] TRANSFORM product_value to product_id
   order_id    order_date  customer_id  ... quantity item_price product_id
0   1730350  Dec 11, 2019        13447  ...       24     113000      P1374
1   1677490  Jul 31, 2019            0  ...        1    1164000      P1370
2   1704211  Oct 18, 2019        16128  ...       12     747000      P1679
3   1679695  Aug 07, 2019        16225  ...        6     590000      P1708
4   1679080  Aug 05, 2019            0  ...        2     740000      P1201

[5 rows x 9 columns]


In [4]:
# 4. Tranform order_date to value with format "YYYY-mm-dd"
print("\n[4] TRANSFORM order_date to FORMAT YYYY-mm-dd")
months_dict = {
   "Jan":"01",
   "Feb":"02",
   "Mar":"03",
   "Apr":"04",
   "May":"05",
   "Jun":"06",
   "Jul":"07",
   "Aug":"08",
   "Sep":"09",
   "Oct":"10",
   "Nov":"11",
   "Dec":"12"
}
df["order_date"] = pd.to_datetime(df["order_date"].apply(lambda x: str(x)[-4:] + "-" + months_dict[str(x)[:3]] + "-" + str(x)[4:7]))
print("    Tipe data:\n", df.dtypes)



[4] TRANSFORM order_date to FORMAT YYYY-mm-dd
    Tipe data:
 order_id                int64
order_date     datetime64[ns]
customer_id             int64
city                   object
province               object
brand                  object
quantity                int64
item_price              int64
product_id             object
dtype: object


In [5]:
# 5. Handling missing value for any column
print("\n[5] HANDLING MISSING VALUE")
# The "city" and "province" columns still have missing values, the missing values ​​in these two columns are filled with "unknown"
df[["city","province"]] = df[["city","province"]].fillna("unknown")
# The brand column also still has a missing value, Change the NaN value to "no_brand"
df["brand"] = df["brand"].fillna("no_brand")
# Check if there are still missing values ​​in all columns 
print("Info:\n", df.info())


[5] HANDLING MISSING VALUE
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_id     5000 non-null   int64         
 1   order_date   5000 non-null   datetime64[ns]
 2   customer_id  5000 non-null   int64         
 3   city         5000 non-null   object        
 4   province     5000 non-null   object        
 5   brand        5000 non-null   object        
 6   quantity     5000 non-null   int64         
 7   item_price   5000 non-null   int64         
 8   product_id   5000 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 351.7+ KB
Info:
 None


In [6]:
# 6. Create a new column "city/province" by combining the "city" column and the "province" column and delete the original column
print("\n[6] Make a new column with name city/province")
df["city/province"] = df["city"] + "/" + df["province"]
# drop the "city" and "province" columns because they have been combined
df.drop(["city","province"], axis=1, inplace=True)
# Checking dataset
print(df.head())


[6] Make a new column with name city/province
   order_id order_date  ...  product_id              city/province
0   1730350 2019-12-11  ...       P1374      Surakarta/Jawa Tengah
1   1677490 2019-07-31  ...       P1370            unknown/unknown
2   1704211 2019-10-18  ...       P1679  Jakarta Pusat/DKI Jakarta
3   1679695 2019-08-07  ...       P1708      Yogyakarta/Yogyakarta
4   1679080 2019-08-05  ...       P1201            unknown/unknown

[5 rows x 8 columns]


In [7]:
# 7. Create a hierarchical index consisting of the columns "city/province", "order_date", "customer_id", "order_id", "product_id"
print("\n[7] MAKE A HIERACHICAL INDEX")
df = df.set_index(["city/province","order_date","customer_id","order_id","product_id"])
# Sort by the new index
df = df.sort_index()
# Checking Dataset
print(df.head())


[7] MAKE A HIERACHICAL INDEX
                                                                     brand  ...  item_price
city/province          order_date customer_id order_id product_id           ...            
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936       BRAND_K  ...      450000
                       2019-11-12 12360       1715116  P0758       BRAND_C  ...      695000
                                                       P3042       BRAND_R  ...      310000
                       2019-12-09 12374       1729036  P1660       BRAND_G  ...     2795000
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628       BRAND_C  ...      695000

[5 rows x 3 columns]


In [9]:
# 8. Create a "total price" column whose formula is multiplication between the "quantity" column and the "item_price" column
print("\n[8] Make a total_price column")
df["total_price"] = df["quantity"] * df["item_price"]
# Checking Dataset
print(df.head())


[8] Make a total_price column
                                                                     brand  ...  total_price
city/province          order_date customer_id order_id product_id           ...             
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936       BRAND_K  ...     10800000
                       2019-11-12 12360       1715116  P0758       BRAND_C  ...      5560000
                                                       P3042       BRAND_R  ...      3720000
                       2019-12-09 12374       1729036  P1660       BRAND_G  ...     11180000
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628       BRAND_C  ...      8340000

[5 rows x 4 columns]


In [10]:
# 9. Slice the dataset so that there is only data for January 2019
print("\n[9] DATASET SLICE FOR JANUARY 2019 ONLY")
idx = pd.IndexSlice
df_jan2019 = df.loc[idx[:, "2019-01-01":"2019-01-31"], :]
print("Final Dataset:\n", df_jan2019)


[9] DATASET SLICE FOR JANUARY 2019 ONLY
Final Dataset:
                                                                      brand  ...  total_price
city/province          order_date customer_id order_id product_id           ...             
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628       BRAND_C  ...      8340000
Bandung/Jawa Barat     2019-01-09 16134       1617055  P1597       BRAND_G  ...      4680000
                       2019-01-10 17392       1617952  P2137       BRAND_M  ...      2124000
                       2019-01-14 15527       1618828  P3115       BRAND_S  ...      1045000
                       2019-01-29 13253       1620289  P0099       BRAND_A  ...      5400000
...                                                                    ...  ...          ...
unknown/unknown        2019-01-30 0           1620766  P3070       BRAND_R  ...       593000
                                                       P3483       BRAND_S  ...      1779000
             